In [ ]:
import os
from PIL import Image
import torch

from diffusers import QwenImageEditPlusPipeline

model_path = "ovedrive/Qwen-Image-Edit-2509-4bit"
pipeline = QwenImageEditPlusPipeline.from_pretrained(model_path, torch_dtype=torch.bfloat16)
print("pipeline loaded") # not true but whatever. do not move to cuda

pipeline.set_progress_bar_config(disable=None)
pipeline.enable_model_cpu_offload()
image1 = Image.open("data/florian_0050.jpg").convert("RGB")
image2 = Image.open("test_bild_shirt.jpg").convert("RGB")
prompt = "Make this person wear the shirt instead of his original clothing. Do not invent new features other then the clothing change. Only replace the original clothing item with the new shirt, while keeping the person exactly the same. keep the pose, body, face, hair, and surroundings exactly the same."
inputs = {
    "image": [image1, image2],
    "prompt": prompt,
    "generator": torch.manual_seed(0),
    "true_cfg_scale": 4.0,
    "negative_prompt": "change pose",
    "num_inference_steps": 20, # even 10 steps should be enough in many cases
}

with torch.inference_mode():
  output = pipeline(**inputs)

output_image = output.images[0]
output_image.save("output_image_flo2.png")
print("image saved at", os.path.abspath("flo_output/output_image_flo2.png"))


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


model_index.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

C:\Users\can-e\PycharmProjects\bat\vton3d-bat\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\can-e\.cache\huggingface\hub\models--ovedrive--Qwen-Image-Edit-2509-4bit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

text_encoder/model-00002-of-00002.safete(…):   0%|          | 0.00/281M [00:00<?, ?B/s]

text_encoder/model-00001-of-00002.safete(…):   0%|          | 0.00/4.81G [00:00<?, ?B/s]